# SQLAlchemy Cookbook

My often-used SQLAlchemy patterns hosted using a Jupyter notebook.

The code is interactive with a PyOdide WASM kernel. You may need to reload the page as occasionally the kernel becomes unresponsive. 

#### Downloading the notebook
Alternatively downlown the notebook and run in your own Python environment:
- Click the "JupyterLab" button above.
- In the file view, right click and select "Download"

In [ ]:
%pip install sqlalchemy

import sqlalchemy

print(sqlalchemy.__version__)

### ORM Configuration
We can override the type annotations in the base type. 

`DateTime` with timezones and string enums can help avoid bugs down the line.

In [ ]:
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column
from sqlalchemy.types import DateTime

from uuid import UUID, uuid4
from datetime import datetime
from enum import Enum


class Base(DeclarativeBase):
    __table_args__ = {'extend_existing': True}  # Don't need this, just to silence any warnings about redefining tables here

    # override default python type mapping
    # https://docs.sqlalchemy.org/en/20/orm/declarative_tables.html#customizing-the-type-map
    type_annotation_map = {
        datetime: DateTime(timezone=True),  # Use timezoned datetime
        Enum: sqlalchemy.Enum(Enum, native_enum=False),  # Use string enum as opposed to native enum types (if available)
    }


In [ ]:
# Ignore some warnings here, don't do this in production code
import warnings
from sqlalchemy.exc import SAWarning
warnings.filterwarnings("ignore", category=SAWarning)


### Reusing column definitions using annotations

Columns are often repeated, for example the id column and datetime columns. Column definitions can be included in the type using [Annotated](https://docs.sqlalchemy.org/en/20/orm/declarative_tables.html#mapping-whole-column-declarations-to-python-types).

In [ ]:
from datetime import UTC, datetime
from typing import Annotated, TypeAlias


now = lambda: datetime.now(UTC)

# NOTE: Mapped column defaults can be a static value or a factory function.
PK: TypeAlias = Annotated[UUID, mapped_column(primary_key=True, default=uuid4)]
Created: TypeAlias = Annotated[datetime, mapped_column(default=now)]
Updated: TypeAlias = Annotated[datetime, mapped_column(default=now, onupdate=now)]

### Creating the ORM Class

In [ ]:
from enum import StrEnum

from sqlalchemy.types import JSON
from sqlalchemy.ext.mutable import MutableList


class UserType(StrEnum):
    STANDARD = "STANDARD"
    ADMIN = "ADMIN"


class User(Base):
    __tablename__ = "user"

    id: Mapped[PK]
    created: Mapped[Created]
    updated: Mapped[Updated]

    # Nullable: by default rows are not nullable
    name: Mapped[str | None] = mapped_column(default=None)

    # JSON type: dialect specefic JSON types may be more powerful.
    groups: Mapped[list[str]] = mapped_column(MutableList.as_mutable(JSON), default=list)

    # Enum
    user_type: Mapped[UserType] = mapped_column(default=UserType.STANDARD)
    

#### Columns types
These are the types I use the most
- `datetime`: main thing to think about is timezones, check if your database supports native timezones, always work in timezones if possible
- `UUID`: I like it over integer for ids, consider using [uuid v7](https://uuid7.com/) if you need it sortable.
- `JSON`: use it instead of JOINs (more on it later) to denormalise data potentially increase the performance. Mutable tracking is pretty much a must here unless you are really sure that you don't plan on mutating the JSON data.
- `Enum`: I typically set the type_annotation_map to sotre it as a string in the db. This depends on the database but typically string is more flexible when it comes to migrations.
- `Nullable`: If you're coming from sqlalchemy 1 then the big change here is that `Mapped` types are not nullable by default. Explicitly mark as `Optional` or union with the `None` type.


### Queries

In [ ]:
from sqlalchemy import select, insert
from sqlalchemy.orm import sessionmaker

engine = sqlalchemy.create_engine("sqlite://")
Base.metadata.create_all(engine)
Session = sessionmaker(engine)

with Session.begin() as session:  # transaction with automatic commit at the exit
    session.add(User(name="Jamie"))  # Insert user as an object, db state will be updated on commit or flush
    session.execute(insert(User).values(name="John"))  # insert the sql way, it's often not needed

with Session() as session:  # Non-transaction session
    user_query = select(User).where(User.name.is_not(None)).order_by(User.name).offset(0).limit(10)
    print(session.execute(user_query).scalars().all())  # Return user objects as an iterator
    print(session.execute(user_query.where(User.name == "Jamie")).scalar_one())  # Return a single user object, error if not found or more than one returned
    print(session.execute(user_query.where(User.name == "Jack")).scalar_one_or_none())  # Return None if not found
    print([user.name for user in session.execute(user_query).scalars()])

#### Inspect statements
Sometimes it's useful to inspect the compiled statements. 

In [ ]:
print(select(User).where(User.name.in_(["Jamie", "Chang"])).compile(compile_kwargs={"literal_binds": True}))

### Foreign Keys and Relationship



In [ ]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship


class Team(Base):
    __tablename__ = "team"

    id: Mapped[PK]
    name: Mapped[str]
    players: Mapped[list['Player']] = relationship()


class Player(Base):
    __tablename__ = "player"

    id: Mapped[PK]
    team_id: Mapped[UUID] = mapped_column(ForeignKey("team.id"))
    team: Mapped[Team] = relationship()


# Note by default reationship is not loaded
print(select(Team))
print(select(Player))
    

### Lazy/Eager loading
By default when we select an object all columns are loaded. We can change this behaviour in the orm.

In [ ]:
class Page(Base):
    __tablename__ = "page"

    id: Mapped[PK]
    name: Mapped[str] = mapped_column()
    data: Mapped[dict] = mapped_column(JSON, deferred=True)

In [ ]:
print(select(Page))  # data is not selected

#### Eagerly load deferred columns

In [ ]:
from sqlalchemy.orm import undefer
print(select(Page).options(undefer(Page.data)))

#### Defer columns at query level

In [ ]:
from sqlalchemy.orm import defer
print(select(Page).options(defer(Page.name)))

### Eagerly load a relationship

Relationships are the biggest cause of N+1 problems. Suppose you want to load a list of players and their teams. By default if we lazy load then we dispatch one query to load the players and for each of the N players dispatch a query to load the team hence N+1.

There are many [relationship loading techniques](https://docs.sqlalchemy.org/en/20/orm/queryguide/relationships.html#relationship-loader-api). I've picked joinedload here for easier demonstration but [selectinload](https://docs.sqlalchemy.org/en/20/orm/queryguide/relationships.html#sqlalchemy.orm.selectinload) is another one I like using.

In [ ]:
from sqlalchemy.orm import joinedload
print(select(Player).options(joinedload(Player.team)))